# Calling the Rewiring America Health Impacts API from Python

In December, Rewiring America released our second public API. The API is based on data and models
we built to type to answer questions about how the health of people across the county would be
affected if communities were to electrify their homes in various ways. 

You can read about the research on [our web site](https://www.rewiringamerica.org/research/home-electrification-health-benefits),
and in the [New York Times](https://www.nytimes.com/2024/12/10/climate/heat-pumps-savings.html).
Reports and new coverage are awesome, but we also wanted to make sure it
was easy for researchers to get the underlying data, sliced and diced to meet their needs. So we
created an API.

In this notebook we are going to demonstrate how you can call the API directly from Python and what
kinds of questions you can ask it.

## Python Environment Setup

The Rewiring America APIs can be called from any Python 3.8+ environment using the requests package. The Health Impacts AP returns data in tabular form, so it is also useful to have Pandas installed in your virtual environment. If you don’t already have both of these packages installed, you can install them with

```shell
pip install requests
```

and

```shell
pip install pandas
```

Those should be the only things you need to ensure are installed before you can run the code in this notebook.

## Imports and configuration

Once we have our environment set up, our next step is to import the packages we are going to use

In [1]:
import requests
import pandas as pd

We also need the URL of the Health Effects API.

In [2]:
#URL = "https://api.rewiringamerica.org/api/v1/health-impacts"
URL = "http://127.0.0.1:8080/api/v1/health-impacts/"

## Our First API Call

### Payload

We are going to start out with a basic call to the API, and then we will build up from there.

In our first call, we will pass in 

- a single metric (premature mortality),
- a single upgrade (a medium efficiency heat pump),
- a single state (Wisconsin).

The return value will be the nationwide impact on premature mortality in person-years if every home
in Wisconsin were to upgrade to a heat pump. Note that the effects are nationwide, because winds can
blow pollutants into nearby states. But the bulk of the effect will occur in and near the state of Wisconsin.

In [3]:
payload = {
    "metrics": "premature_mortality_incidence_delta",
    "upgrade": "med_eff_hp_hers_sizing_no_setback",
    "state": "WI"
}

### Headers

In addition to the payload, we will send in some standard headers with each call, to indicate
that we are sending the payload in JSON form and we expect the results to come back in JSON.

In [4]:
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
}

### Make the Call

Now we can make the actual call, look at the response code, which should be 200 to indicate
that everything was OK, and then the text of the response.

In [5]:
# Make the API call:
response = requests.post(URL, json=payload, headers=headers)

In [6]:
response.status_code

200

In [7]:
response.text

'{"data":[{"state":"WI","premature_mortality_incidence_delta":27.999598212086404,"number_of_households":2189591}],"warnings":[]}'

### Data from the Response

Now we will make a data frame out of the JSON data that comes back from the call.

We asked for one state, so the response will have one row. It will also have a column
to indicate what the state is. This will be more important when we ask for more than
one state. We asked for one metric, so it will have a column for that. It will also
have a column for the number of households in the state.

In [8]:
df_data = pd.DataFrame(response.json()["data"])

In [9]:
df_data

,state,premature_mortality_incidence_delta,number_of_households
0,WI,27.999598,2189591


So looking at the results, we see that if all of the households in Wisconsin were to switch to heat pumps, 28 lives per year would be saved.

## Expanding our Query to All States

Now let's make a second query. Just like in the first one, we
Wpass in a single metric (premature mortality) and a single upgrade (a
medium efficiency heat pump). But unlike in the Wisconsin example, we do not pass in a state. Therefore, we get results for all states in the continental United States.

In [10]:
payload = {
    "metrics": "premature_mortality_incidence_delta",
    "upgrade": "med_eff_hp_hers_sizing_no_setback"
}

response = requests.post(URL, json=payload, headers=headers)

In [11]:
df_data = pd.DataFrame(response.json()["data"])
df_data

,state,premature_mortality_incidence_delta,number_of_households
0,AL,15.862550,1409929
1,AR,11.267364,884020
2,AZ,2.975578,1848670
3,CA,166.436745,12098560
4,CO,27.328565,1904118
5,CT,43.784559,1290316
6,DC,10.717516,236562
7,DE,11.109927,299274
8,FL,10.920949,4840199
9,GA,38.972264,2898066


Our resulting dataframe has the same columns as before, but since we did not 
ask for a specific state, we got all of them. 

### Deriving New Metrics

Since the data is in a data frame, we can manipulate it a variety of ways. For
example, we could add a column for the ratio of change in mortality to number
of households, then see which states have the highest value for that derived
metric.

In [12]:
df_data["mortality_per_household"] = df_data["premature_mortality_incidence_delta"] / df_data["number_of_households"]

df_data.nlargest(10, "mortality_per_household")

,state,premature_mortality_incidence_delta,number_of_households,mortality_per_household
32,NY,870.082923,6859572,0.000127
29,NJ,204.186709,3061262,0.000067
17,MA,131.983145,2449155,0.000054
37,RI,19.806513,392252,0.000050
18,MD,89.941786,1791043,0.000050
6,DC,10.717516,236562,0.000045
36,PA,199.916981,4489835,0.000045
20,MI,141.640629,3679665,0.000038
7,DE,11.109927,299274,0.000037
5,CT,43.784559,1290316,0.000034
